In [1]:
import os
from glob import glob # 파일 경로 패턴 매칭에 유용

from dotenv import load_dotenv
load_dotenv()

True

## 1. LangChain RAG 구현

- RAG(Retrieval Augmented Generation)는 쉽게 말해, LLM이 **외부 데이터를 참고해서** 더 똑똑한 답변을 생성하도록 돕는 기술
- LLM의 고질적인 문제인 **환각 현상(Hallucination)을 줄이고**, **최신 정보나 특정 도메인 지식을 활용**할 수 있게 해주며, LLM한테 오픈북 시험 보게 하는 거랑 비슷함.

크게 다음과 같은 단계를 거침:
1.  **문서 로드(Load):** RAG의 재료가 될 외부 데이터(텍스트 파일, PDF, 웹페이지 등)를 불러옴.
2.  **분할(Split):** 불러온 문서를 LLM이 한 번에 처리하기 좋은 크기(청크)로 잘게 쪼개며, 너무 길면 LLM의 이슈 발생.
3.  **저장(Store):** 쪼갠 문서 조각(청크)들을 벡터 임베딩으로 변환해서 벡터 저장소에 넣어둠.
4.  **검색(Retrieve):** 사용자 질문이 들어오면, 질문과 의미적으로 유사한 문서 청크들을 벡터 저장소에서 찾아옴.
5.  **생성(Generate):** 찾아온 문서 청크들을 참고자료 삼아 LLM이 최종 답변을 생성함. 이때 질문과 참고자료를 LLM에게 잘 전달하는 게 중요함.

### 1.1 문서 로더 (Document Loaders)

- 첫 단계는 RAG 시스템의 지식 창고가 될 문서들을 불러오는 거임. 
LangChain은 다양한 형식의 문서를 로드할 수 있는 여러 `DocumentLoader`를 제공함. 
(예: `TextLoader`, `PyPDFLoader`, `CSVLoader`, `UnstructuredURLLoader` 등)

**팁:**
- 디렉터리 자체 로더도 유용하게 쓰일것 같음.

In [2]:
# 'data' 폴더 안에 있는 모든 '_KR.txt' 파일을 찾음
txt_files = glob(os.path.join('../data', '*_KR.txt'))

print(txt_files)

['../data\\Rivian_KR.txt', '../data\\Tesla_KR.txt']


In [3]:
# 첫 번째 텍스트 파일 내용 확인 (샘플 로드)
from langchain_community.document_loaders import TextLoader

# TextLoader는 지정된 텍스트 파일을 로드함.
loader_sample = TextLoader(txt_files[0], encoding='utf-8') 
data_sample = loader_sample.load() # load() 메서드는 Document 객체의 리스트를 반환함.

# 로드된 Document 객체의 개수 확인. TextLoader는 파일 하나당 하나의 Document 객체를 생성함.
print(f"로드된 Document 객체 수 (샘플): {len(data_sample)}")
print(f"data_sample의 타입: {type(data_sample)}")

로드된 Document 객체 수 (샘플): 1
data_sample의 타입: <class 'list'>


In [4]:
# 로드된 Document 객체의 내용(page_content)과 메타데이터(metadata)를 살펴봄.
# data_sample[0]은 첫 번째 Document 객체를 의미함.
# .page_content 속성에는 파일의 실제 텍스트 내용이 담겨있음.
print("--- 첫 번째 Document 내용 (page_content) ---")
print(data_sample[0].page_content)
print("\n--- 첫 번째 Document 메타데이터 (metadata) ---")
# .metadata 속성에는 문서의 출처(source) 같은 부가 정보가 딕셔너리 형태로 저장됨.
print(data_sample[0].metadata)

--- 첫 번째 Document 내용 (page_content) ---
리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임. 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음.

--- 첫 번째 Document 메타데이터 (metadata) ---
{'source': '../data\\Rivian_KR.txt'}


In [5]:
from tqdm import tqdm 

all_documents = [] # 로드된 Document 객체들을 담을 리스트

for text_file in tqdm(txt_files, desc="문서 로딩 중"):
    loader = TextLoader(text_file, encoding='utf-8')
    all_documents.extend(loader.load())

# 총 몇 개의 Document가 로드되었는지 확인. 파일 개수만큼 생성됨.
print(f"\n총 로드된 Document 수: {len(all_documents)}")
print(f"첫 번째 로드된 문서의 출처: {all_documents[0].metadata['source']}")
print(f"두 번째 로드된 문서의 출처: {all_documents[1].metadata['source']}")
print(f"로드된 각 항목의 타입: {type(all_documents[0])}")

문서 로딩 중: 100%|██████████| 2/2 [00:00<00:00, 1999.67it/s]


총 로드된 Document 수: 2
첫 번째 로드된 문서의 출처: ../data\Rivian_KR.txt
두 번째 로드된 문서의 출처: ../data\Tesla_KR.txt
로드된 각 항목의 타입: <class 'langchain_core.documents.base.Document'>


### 1.2 텍스트 분할기 (Text Splitters)
- 개인적 의견으로는 모델의 성능이외에 가장 중요한 요소라고 판다.
- LLM은 한 번에 처리할 수 있는 텍스트 양(토큰 수)에 제한이 있음 (컨텍스트 윈도우라고 부름). 
- 그래서 긴 문서를 통째로 넣기보다는, 의미적으로 관련된 작은 조각(청크)으로 나눠서 처리하는 게 일반적임.
- 텍스트 분할기는 이 역할을 담당함. 어떻게 나누느냐에 따라 RAG 성능에 영향을 줄 수 있으니 중요함!

**주요 파라미터 (CharacterTextSplitter 기준):**
- `chunk_size`: 나눌 청크의 최대 크기 (글자 수 기준). LLM의 컨텍스트 윈도우 크기와 데이터 특성을 고려해서 정해야 함.
- `chunk_overlap`: 청크끼리 겹치는 글자 수. 청크 경계에서 의미가 잘리는 걸 방지하고 문맥 연속성을 유지하는 데 도움을 줌. 너무 크면 중복 데이터가 많아지고, 작으면 문맥 파악이 어려울 수 있음.
- `separator`: 청크를 나눌 기준이 되는 구분자. 기본값은 `"\n\n"`(연속된 개행 문자, 보통 문단 구분)임. 데이터 특성에 맞춰 변경 가능함. 예를 들어 정규 표현식을 사용하여 특정 조건에서 분할을 할 수 있음.(예: 문장 단위로 나누고 싶다면 마침표 `.` 등을 고려해볼 수 있으나, `RecursiveCharacterTextSplitter`가 더 적합할 수 있음)

**장점:**
- LLM의 컨텍스트 윈도우 제약에 맞춰 문서를 효과적으로 처리할 수 있음.
- `chunk_overlap`을 통해 문맥 손실을 최소화하려는 시도를 함.

**단점:**
- 최적의 `chunk_size`와 `chunk_overlap`을 찾기 위해 실험이 필요할 수 있음.
- 단순한 `CharacterTextSplitter`는 의미 단위보다는 글자 수 기준으로 나누므로, 문맥이 어색하게 잘릴 가능성이 여전히 존재함. (이를 보완하기 위해 `RecursiveCharacterTextSplitter`나 특정 언어/포맷에 맞는 스플리터 사용을 고려)

**팁:**
- `CharacterTextSplitter`는 가장 기본적인 분할기임. 좀 더 똑똑하게 나누고 싶다면 `RecursiveCharacterTextSplitter`를 고려해볼 수 있음 (문단 -> 문장 -> 단어 순으로 계층적 분할 시도).
- `chunk_size`와 `chunk_overlap`은 사용하는 LLM 모델, 데이터의 특성(평균 문단/문장 길이 등)을 고려하여 실험을 통해 최적값을 찾는 게 좋음.

In [6]:
# 분할하기 전에 각 문서의 원본 글자 수를 확인.
char_counts = [len(doc.page_content) for doc in all_documents]
for i, count in enumerate(char_counts):
    print(f"문서 {i+1} ({all_documents[i].metadata['source']})의 글자 수: {count}")

문서 1 (../data\Rivian_KR.txt)의 글자 수: 350
문서 2 (../data\Tesla_KR.txt)의 글자 수: 439


In [7]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=200,         
    chunk_overlap=100,      
    separator='\n',      # 개행문자("\n")를 기준으로 우선 분할 시도 
    # length_function=len, # 청크 길이를 계산하는 함수 (기본값은 len)
    # is_separator_regex=False # separator가 정규표현식인지 여부 (기본값은 False)
)

# Document 객체 리스트를 받아 분할된 Document 청크 리스트를 반환함
splitted_texts = text_splitter.split_documents(all_documents)

print(f"생성된 텍스트 청크 수: {len(splitted_texts)}")
chunk_lengths = [len(text.page_content) for text in splitted_texts]
print(f"각 청크의 길이: {chunk_lengths}")

# 출력에 종종 보이는 "Created a chunk of size ... which is longer than the specified ..." 경고는 
# separator로 나눈 덩어리가 chunk_size보다 클 때 발생함. 
# CharacterTextSplitter는 separator를 우선으로 고려하고, 그 다음 chunk_size를 맞추려고 하기 때문임.
# 만약 separator로 나눈 가장 작은 단위 자체가 chunk_size보다 크면 어쩔 수 없음.

print("\n--- 첫 번째 청크 내용 ---")
print(splitted_texts[0].page_content)
print(f"출처: {splitted_texts[0].metadata['source']}")

print("\n--- 두 번째 청크 내용 (겹치는 부분 확인) ---")
print(splitted_texts[1].page_content)
print(f"출처: {splitted_texts[1].metadata['source']}")

생성된 텍스트 청크 수: 7
각 청크의 길이: [143, 170, 176, 113, 196, 193, 149]

--- 첫 번째 청크 내용 ---
리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
출처: ../data\Rivian_KR.txt

--- 두 번째 청크 내용 (겹치는 부분 확인) ---
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임. 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
출처: ../data\Rivian_KR.txt


**`CharacterTextSplitter`의 동작 방식 및 경고 메시지 이해:**

- 위 코드에서 `chunk_size`를 설정했지만, 실제 생성된 청크가 이보다 클 수 있음.
- `CharacterTextSplitter`는 먼저 `separator`를 기준으로 텍스트를 나눔.
- 이렇게 나눠진 덩어리가 `chunk_size`보다 크면, 그 덩어리 안에서 다시 `chunk_size`에 맞춰 자르려고 시도함.
- 하지만 `separator`로 나눈 가장 작은 단위(예: 한 문단) 자체가 `chunk_size`를 초과하면, 어쩔 수 없이 `chunk_size`보다 큰 청크가 생성될 수 있음. (위 예제에서는 `separator='\n'`이므로 한 줄 자체가 길면 그럴 수 있음)
- 이게 바로 `Created a chunk of size ... which is longer than the specified ...` 경고가 뜨는 이유임. (샘플이 짧으면면 안 뜰 수도 있음)

**Q. `separator`를 빈 문자열(`''`)로 하면 어떻게 될까?**
- `separator=''`로 설정하면, 각 문자를 기준으로 나누려고 함. 
- 이렇게 되면 `chunk_size`에 거의 정확히 맞춰서 잘리겠지만, 단어나 문장의 의미가 완전히 깨질 수 있음.
- 대부분의 경우 권장하지 않는 방식임. 문맥 유지가 중요하기 때문!

### 1.3 임베딩 모델 (Embeddings)

텍스트를 컴퓨터가 이해하고 비교할 수 있는 숫자 표현, 즉 **벡터(vector)**로 바꾸는 과정을 '임베딩'이라고 함. 
쉽게 말해, 단어나 문장을 숫자로 된 좌표에 찍는 거임. 이때 의미가 비슷한 단어나 문장은 벡터 공간에서 가까운 곳에 위치하게 됨. 
RAG에서는 이 임베딩을 사용해서 사용자 질문과 가장 관련 있는 문서 청크를 찾아냄. (수학적으로 유사도를 계산할 수 있게 되니까!)

LangChain은 OpenAI, HuggingFace Hub (오픈소스 모델들), Cohere 등 다양한 임베딩 모델을 지원함. 
여기서는 OpenAI의 `text-embedding-3-small` 모델을 사용해볼 거임. 가성비가 좋은 편임.

**임베딩 모델 선택 시 고려사항:**
- **성능:** 얼마나 의미를 잘 파악해서 벡터로 표현하는가? (이게 제일 중요)
- **비용:** API 호출 비용 (OpenAI 등) 또는 자체 호스팅/실행 비용 (오픈소스 모델).
- **속도:** 임베딩 생성 속도. 문서가 많으면 중요해짐.
- **차원 수:** 생성되는 벡터의 차원. 차원이 높을수록 더 많은 정보를 담을 수 있지만, 계산량이 늘어나고 저장 공간도 더 필요할 수 있음. (예: `text-embedding-3-small`은 1536차원, `text-embedding-3-large`는 3072차원)
- **지원 언어:** 주로 다루는 데이터의 언어를 잘 지원하는지 확인해야 함.

**장점 (OpenAI Embeddings 사용 시):**
- 비교적 높은 성능의 임베딩을 별도 모델 관리 없이 쉽게 API로 사용 가능함.
- 다양한 언어를 잘 지원하는 편임.

**단점 (OpenAI Embeddings 사용 시):**
- API 호출 비용 발생. (문서가 많거나 자주 임베딩하면 비용이 꽤 나옴)
- 인터넷 연결이 필수임.
- 데이터가 OpenAI 서버로 전송됨 (민감 데이터의 경우 내부 정책/규정 확인 필요).

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI의 임베딩 모델 중 하나. 더 큰 모델도 있음 (예: text-embedding-3-large)
    # dimensions=512 # 일부 최신 모델은 요청 차원 수 지정 가능 (비용 절감 효과). 기본값 사용.
)

# 샘플 텍스트를 임베딩해봄
sample_text_for_embedding = "테슬라 창업자는 누구인가요?"
vector = embeddings_model.embed_query(sample_text_for_embedding) # 단일 텍스트(주로 쿼리) 임베딩은 embed_query 사용
# 여러 텍스트(문서 청크들)를 한 번에 임베딩하려면 embed_documents 사용 (다음 단계인 벡터 저장소에서 내부적으로 호출됨)

print(f"임베딩 벡터의 차원: {len(vector)}")
print(f"임베딩 벡터 (앞 5개 값): {vector[:5]}") # 실제 벡터 값은 매우 긴 숫자 리스트

임베딩 벡터의 차원: 1536
임베딩 벡터 (앞 5개 값): [-0.03408345580101013, -0.009563403204083443, 0.005879570730030537, 0.01511972863227129, 0.001523038954474032]


### 1.4 벡터 저장소 (Vector Stores)

임베딩된 텍스트 청크(벡터)들을 저장하고, 나중에 유사도 기반으로 빠르게 검색할 수 있게 해주는 특수한 데이터베이스임. 
일반 DB랑은 좀 다름. 벡터 간의 거리(유사도) 계산에 최적화되어 있음.
LangChain은 Chroma, FAISS, Pinecone, Weaviate, PGVector 등 다양한 벡터 저장소를 지원함.
여기서는 사용하기 간편한 인메모리(또는 로컬 디스크 기반) 벡터 저장소인 `Chroma`를 사용해볼 거임.

**Chroma 사용법:**
- `from_documents()`: 분할된 텍스트 청크(`splitted_texts`)와 임베딩 모델(`embeddings_model`)을 전달하면, 내부적으로 각 청크를 임베딩하고 Chroma DB에 저장함.
- `persist_directory`: 지정된 경로에 DB를 저장하여 나중에 재사용할 수 있게 함. 지정 안 하면 인메모리로만 동작해서 프로그램 끝나면 사라짐.
- `collection_name`: DB 내에서 특정 데이터셋을 구분하는 이름. (선택 사항)

**장점 (Chroma 사용 시):**
- 설치 및 사용이 매우 간편함. 로컬 환경에서 테스트하거나 소규모 프로젝트에 적합.
- 디스크에 저장하여 영속성 확보 가능 (`persist_directory` 사용 시).
- 별도 서버 구축 없이 바로 사용 가능.

**단점 (Chroma 사용 시):**
- 대규모 데이터(수백만 건 이상)나 매우 높은 QPS(초당 쿼리 수) 환경에서는 전문적인 벡터 DB(예: Pinecone, Weaviate, Milvus 등)보다 성능이나 확장성이 떨어질 수 있음.
- 고급 기능(세밀한 필터링, 실시간 업데이트 등)이 제한적일 수 있음.

In [9]:
# 각 청크는 Document 객체 형태를 유지하고 있음 (page_content와 metadata 가짐)
print("--- 벡터 저장소에 전달될 첫 번째 청크 ---")
print(splitted_texts[0])

--- 벡터 저장소에 전달될 첫 번째 청크 ---
page_content='리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.' metadata={'source': '../data\\Rivian_KR.txt'}


In [10]:
from langchain_chroma import Chroma

CHROMA_DB_PATH = "./chroma_db_rag_tutorial"

# Chroma 벡터 저장소 생성 또는 로드
# splitted_texts ("분할된 Document 청크들")와 embeddings_model (임베딩 생성기)을 전달
vectorstore = Chroma.from_documents(
    documents=splitted_texts,      # 임베딩하고 저장할 Document 객체 리스트
    embedding=embeddings_model,    # 텍스트를 벡터로 변환할 임베딩 모델
    collection_name="rag_tutorial_collection", # 벡터 컬렉션 이름 (나중에 불러올 때 사용 가능)
    persist_directory=CHROMA_DB_PATH # 벡터 데이터를 저장할 디렉토리. 지정하면 디스크에 저장됨.
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")
print(f"Chroma DB가 저장된 경로: {CHROMA_DB_PATH}")

벡터 저장소에 저장된 문서 수: 28
Chroma DB가 저장된 경로: ./chroma_db_rag_tutorial


**벡터 저장소에서 유사도 검색하기:**

벡터 저장소의 핵심 기능은 특정 쿼리(질문)가 주어졌을 때, 저장된 벡터들과의 유사도를 계산하여 가장 관련 높은 문서(청크)들을 찾아주는 것임.
`similarity_search()` 메서드를 사용하면 됨.
- `query`: 검색할 질문 텍스트.
- `k`: 반환받을 유사 문서의 개수 (기본값은 보통 4). 

이 메서드는 내부적으로 다음 단계를 수행함:
1.  입력된 `query`를 (우리가 제공한) 임베딩 모델을 사용해 벡터로 변환.
2.  변환된 쿼리 벡터와 벡터 저장소에 있는 모든 문서 청크 벡터들 간의 유사도(보통 코사인 유사도)를 계산.
3.  유사도가 높은 순으로 `k`개의 문서 청크를 반환함.

In [11]:
# 벡터 저장소에서 "테슬라 창업자는 누구인가요?" 와 유사한 문서 청크를 검색해봄.
query_for_search = "테슬라 창업자는 누구인가요?"
result_docs_similarity = vectorstore.similarity_search(
    query_for_search,
    k=2  # 가장 유사한 문서 2개를 가져옴
)

print(f"'similarity_search' 결과 문서 수: {len(result_docs_similarity)}")
for i, doc in enumerate(result_docs_similarity):
    print(f"\n--- 검색된 문서 {i+1} ---")
    print(doc.page_content)
    print(f"(출처: {doc.metadata['source']})")

'similarity_search' 결과 문서 수: 2

--- 검색된 문서 1 ---
테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.
(출처: ../data\Tesla_KR.txt)

--- 검색된 문서 2 ---
테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.
(출처: ../data\Tesla_KR.txt)


### 1.5 검색기 (Retrievers)

검색기는 벡터 저장소 같은 소스에서 특정 쿼리에 관련된 문서를 가져오는 역할을 하는 LangChain의 표준 인터페이스임.
벡터 저장소 자체도 `similarity_search` 같은 메서드를 제공하지만, 검색기 인터페이스(`as_retriever()`)를 사용하면 LangChain의 다른 컴포넌트(특히 체인)와 더 매끄럽게 결합할 수 있음. 일종의 규격화된 부품이라고 생각하면 됨.

`as_retriever()`를 사용하면 벡터 저장소를 간단히 검색기로 변환할 수 있음.

**주요 파라미터 (`as_retriever()`의 `search_kwargs` 내):**
- `k`: 검색할 문서의 개수를 지정. `similarity_search`의 `k`와 동일한 역할.
- `score_threshold`: 특정 유사도 점수 이상의 문서만 반환하도록 필터링 가능. (0~1 사이 값, 높을수록 엄격. Chroma는 코사인 유사도가 아닌 거리 기반 점수를 사용할 수 있어 값 범위가 다를 수 있음. 확인 필요)
- `fetch_k`: (MMR 같은 고급 검색 전략 사용 시) 유사도 계산 전에 내부적으로 가져올 문서의 개수. 이 중에서 최종 `k`개를 선택함.

**장점:**
- LangChain 체인과의 통합이 용이함.
- 단순 유사도 검색 외에 MMR(Maximal Marginal Relevance, 다양성 고려), Self-Query (쿼리 자체를 분석해 필터링 조건 생성) 등 다양한 고급 검색 전략을 적용하기 쉬움.

**단점:**
- Retriever의 종류와 설정이 다양해서, 상황에 맞는 최적의 것을 고르려면 학습과 실험이 필요함.
- 복잡한 검색 전략은 추가적인 LLM 호출이나 계산을 요구할 수 있음.

**팁:**
- 단순히 유사도 높은 문서만 가져오는 것 외에도, 다양한 검색 전략을 가진 검색기들이 있음. (예: `MultiQueryRetriever`, `ParentDocumentRetriever`, `SelfQueryRetriever` 등) 상황에 맞게 선택하면 RAG 성능을 더 끌어올릴 수 있음.

In [12]:
# 벡터 저장소를 Retriever로 변환. 
# search_kwargs를 통해 검색 시 옵션(예: 가져올 문서 개수 k)을 설정할 수 있음.
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) # 상위 2개 문서만 가져오도록 설정

# retriever.invoke() 메서드로 문서 검색 실행
relevant_docs_retriever = retriever.invoke(query_for_search) # query_for_search는 이전 셀에서 정의("테슬라 창업자는 누구인가요?")


print(f"Retriever가 검색한 관련 문서 수: {len(relevant_docs_retriever)}")
for i, doc in enumerate(relevant_docs_retriever):
    print(f"\n--- Retriever 검색 문서 {i+1} ---")
    print(doc.page_content)
    print(f"(출처: {doc.metadata['source']})")

Retriever가 검색한 관련 문서 수: 2

--- Retriever 검색 문서 1 ---
테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.
(출처: ../data\Tesla_KR.txt)

--- Retriever 검색 문서 2 ---
테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.
(출처: ../data\Tesla_KR.txt)


### 1.6 언어 모델 (LLMs)

RAG의 'G'(Generation, 생성)를 담당하는 핵심 엔진, 바로 대규모 언어 모델(LLM)임.
검색기가 찾아온 관련 문서를 참고자료(Context) 삼아, 사용자 질문에 대한 최종 답변을 생성함.
LangChain은 OpenAI의 GPT 모델들, HuggingFace의 오픈소스 모델, Anthropic의 Claude 등 다양한 LLM을 지원함.

여기서는 OpenAI의 `gpt-4o-mini` 모델을 사용해볼 거임.

**주요 파라미터 (ChatOpenAI 기준):**
- `model` (또는 `model_name`): 사용할 LLM 모델 이름. (예: `gpt-4o-mini`, `gpt-4-turbo`, `gpt-3.5-turbo`)
- `temperature`: 생성될 텍스트의 무작위성(창의성) 조절. 0에 가까울수록 결정적이고 일관된 답변, 높을수록 다양하고 창의적인 답변이 나옴. 보통 0.0 ~ 2.0 사이 값을 사용. RAG에서는 사실 기반 답변이 중요하므로 낮게 설정하는 경우가 많음 (예: 0 또는 0.1).
- `max_tokens`: 생성할 최대 토큰 수. 답변 길이를 제어함. 너무 작으면 답변이 중간에 잘릴 수 있고, 크면 비용이 증가하고 응답 시간이 길어질 수 있음.

**장점 (OpenAI LLMs 사용 시):**
- 일반적으로 강력한 언어 이해 및 생성 능력을 보여줌.
- 다양한 크기와 성능의 모델을 선택할 수 있어 유연함.
- API 형태로 제공되어 모델 관리 부담이 적음.

**단점 (OpenAI LLMs 사용 시):**
- API 호출 비용 발생. (토큰 사용량에 따라 과금)
- 인터넷 연결 필수.
- 데이터가 OpenAI 서버로 전송됨 (민감 데이터 처리 시 주의).
- 가끔 예상치 못한 답변(환각)을 생성할 수 있으나, RAG가 이를 완화하는 데 도움을 줌.

**LLM 사용 시 주의점 (특히 RAG에서):**
- **환각(Hallucination) 현상:** LLM이 학습 데이터에 없거나 잘못된 정보를 마치 사실인 것처럼 생성하는 현상. RAG는 이 문제를 줄이는 데 큰 도움을 주지만, 100% 막을 순 없음.
- **컨텍스트 주입의 중요성:** 검색된 관련 문서를 LLM 프롬프트에 효과적으로 잘 넣어줘야 함. 이것이 RAG의 핵심! 프롬프트 엔지니어링이 중요해짐.

In [13]:
# 1. Context를 제공하지 않고 LLM에게 질문했을 때의 답변 (환각 발생 가능성 테스트)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,        # 답변의 일관성을 위해 0으로 설정 (창의성 낮춤)
    max_tokens=150,       # 답변 최대 길이를 150 토큰으로 제한
    )

response_without_context = llm.invoke(query_for_search) # query_for_search는 "테슬라 창업자는 누구인가요?"
print(f"--- LLM 응답 (컨텍스트 없음) ---")
print(response_without_context.content)
# 위 응답을 보면, 우리 데이터에는 없는 일반적인 웹 지식을 바탕으로 답하거나, 
# 혹은 정보가 부족하면 부정확한 답변을 할 수 있음. (예: 일론 머스크를 창업자로 바로 언급)

--- LLM 응답 (컨텍스트 없음) ---
테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다. 이후 그는 테슬라의 가장 잘 알려진 얼굴이 되었습니다.


**컨텍스트(Context)를 활용한 답변 생성:**

이제 RAG의 핵심! 앞서 Retriever가 검색한 관련 문서(`relevant_docs_retriever`)를 LLM에게 참고자료로 함께 제공하여 질문에 답변하도록 해보겠음.
이렇게 하면 LLM은 주어진 컨텍스트 내에서 정보를 찾아 답변하므로, 훨씬 정확하고 우리가 제공한 데이터에 기반한 신뢰도 높은 결과를 얻을 수 있음.

In [14]:
# 2. Context를 제공하고 LLM에게 질문했을 때의 답변 (수동 프롬프트 구성)

# 검색된 문서들의 내용을 하나의 문자열로 합침
context_for_llm_manual = "\n\n".join([doc.page_content for doc in relevant_docs_retriever])

# LLM에게 전달할 프롬프트를 구성. 
# 컨텍스트를 명확히 제시하고, 해당 컨텍스트를 기반으로 답변하도록 지시하는 것이 중요.
prompt_with_context = f"""주어진 컨텍스트 정보를 바탕으로 다음 질문에 답변해주세요.
만약 컨텍스트에 답변이 없다면, "컨텍스트에 정보가 없습니다."라고 답변해주세요.

컨텍스트:
----------
{context_for_llm_manual}
----------

질문: {query_for_search}

답변: 
"""

In [15]:
print("--- LLM에게 전달될 최종 프롬프트 (수동 구성) ---")
print(prompt_with_context)

--- LLM에게 전달될 최종 프롬프트 (수동 구성) ---
주어진 컨텍스트 정보를 바탕으로 다음 질문에 답변해주세요.
만약 컨텍스트에 답변이 없다면, "컨텍스트에 정보가 없습니다."라고 답변해주세요.

컨텍스트:
----------
테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.

테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.
----------

질문: 테슬라 창업자는 누구인가요?

답변: 



In [16]:
response_with_context_manual = llm.invoke(prompt_with_context)
print("\n--- LLM 응답 (컨텍스트 활용, 수동 프롬프트) ---")
print(response_with_context_manual.content)
# 컨텍스트를 제공하니, 우리 데이터에 있는 '마틴 에버하드와 마크 타페닝'을 정확히 찾아 답변하거나, 관련 정보를 바탕으로 답함!


--- LLM 응답 (컨텍스트 활용, 수동 프롬프트) ---
테슬라의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.


### 1.7 전체 RAG 파이프라인 구성 (Chains)

지금까지 개별 컴포넌트(로더, 스플리터, 임베딩, 벡터 저장소, 리트리버, LLM)들을 하나씩 살펴봤음.
LangChain의 진가는 이런 컴포넌트들을 '체인(Chain)'으로 엮어 하나의 파이프라인으로 자동화할 수 있음.
LangChain Expression Language (LCEL, `|` 파이프 연산자 사용)을 사용하면 파이프라인을 더 직관적이고 유연하게 구성할 수 있음.

**주요 체인 구성 요소:**
- **프롬프트 템플릿 (`ChatPromptTemplate`):** LLM에게 전달할 프롬프트의 틀을 정의함. 컨텍스트(`{context}`)와 사용자 질문(`{input}` 또는 `{question}`) 같은 변수를 포함할 수 있음.
  - `hub.pull()`: LangChain Hub에 공유된 검증된 프롬프트 템플릿을 쉽게 가져와 사용할 수 있음. (옵션1)
  - `ChatPromptTemplate.from_template()` 또는 `ChatPromptTemplate.from_messages()`: 직접 프롬프트 문자열이나 메시지 목록을 작성하여 템플릿을 만듦. (옵션2)
- **`create_stuff_documents_chain`:** 검색된 여러 문서(청크)들을 하나의 컨텍스트로 "stuffing"(채워넣기)하여 LLM 프롬프트에 전달하고, LLM으로부터 답변을 받는 체인을 생성함. 
  - **장점 (Stuffing):** 구현이 쉽고, 모든 검색된 정보를 LLM이 한 번에 볼 수 있어 종합적인 답변에 유리할 수 있음.
  - **단점 (Stuffing):** 검색된 문서들의 총 길이가 LLM의 컨텍스트 윈도우 제한을 넘으면 에러 발생 또는 내용 잘림. 문서가 많을수록 비용과 처리 시간 증가.
  - (다른 문서 결합 방식: `map_reduce`, `refine`, `map_rerank` 등. 문서가 매우 많거나 길 때, 또는 컨텍스트 윈도우가 작을 때 고려)
- **`create_retrieval_chain`:** 사용자 질문을 받아 Retriever로 관련 문서를 검색하고, 검색된 문서를 `create_stuff_documents_chain` 같은 문서 결합 체인에 전달하여 최종 답변을 생성하는 전체 RAG 흐름을 만드는 체인임. 얘가 RAG의 핵심 로직을 편하게 만들어줌.

**흐름 요약 (create_retrieval_chain 사용 시):**
1.  사용자 질문 (`input`)이 들어옴.
2.  `create_retrieval_chain`은 이 `input`을 지정된 Retriever에게 전달.
3.  Retriever는 유사한 문서들을 검색하여 `context`로 만듦.
4.  원래 `input`과 방금 검색된 `context`는 `combine_docs_chain` (여기서는 `create_stuff_documents_chain`으로 생성된 것)으로 전달됨.
5.  `combine_docs_chain`은 프롬프트 템플릿에 `input`과 `context`를 채워 넣어 LLM을 호출.
6.  LLM이 생성한 답변(`answer`)을 최종 결과로 반환 (이때 `input`과 `context`도 함께 반환되어 과정을 살펴보기 좋음).

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain # Retriever와 문서결합 체인을 연결
from langchain.chains.combine_documents import create_stuff_documents_chain # 문서를 "stuff" 방식으로 결합

# --- 프롬프트 설정 --- #
# LangChain Hub에서 프롬프트를 가져오는 대신 직접 작성해봄. (hub.pull도 좋음!)
# {context} 에는 검색된 문서 내용이, {input} 에는 사용자 질문이 들어감.
rag_prompt_template = ChatPromptTemplate.from_template("""당신은 질문에 답변하는 어시스턴트입니다.
제공된 컨텍스트만을 사용하여 사용자의 질문에 답변해주세요.
만약 컨텍스트에서 답변을 찾을 수 없다면, "죄송합니다, 제공된 정보로는 답변을 찾을 수 없습니다."라고 말해주세요.

컨텍스트:
{context}

질문: {input}

답변:""")

# --- 체인 생성 --- #
# 1. 문서 결합 및 LLM 호출 체인 생성 (Stuffing 방식)
# 검색된 문서들(context)과 사용자 질문(input)을 프롬프트에 넣어 LLM을 호출함.
combine_docs_chain = create_stuff_documents_chain(
    llm=llm, # 앞에서 정의한 ChatOpenAI 객체
    prompt=rag_prompt_template
)

# 2. 검색(Retrieval) 체인 생성
# Retriever (앞에서 만든 vectorstore.as_retriever())와 위에서 만든 combine_docs_chain을 연결.
# 이 체인은 먼저 retriever로 문서를 가져오고, 그 결과를 combine_docs_chain에 전달함.
# 여기서 retriever는 이전 셀에서 k=2로 설정한 것을 사용함.
rag_chain = create_retrieval_chain(
    retriever=retriever, # relevant_docs_retriever = retriever.invoke(query) 에서 사용한 그 retriever (k=2)
    combine_docs_chain=combine_docs_chain
)

print("RAG Chain이 성공적으로 생성되었습니다.")

RAG Chain이 성공적으로 생성되었습니다.


In [18]:
# --- 체인 실행 --- #
query_for_chain = "리비안 창업자는 누구이고, 어떤 회사로부터 투자를 받았나요?"

# rag_chain.invoke()의 입력은 딕셔셔너리 형태여야 하며, 프롬프트에 정의된 'input' 키를 사용함.
response_from_chain = rag_chain.invoke({"input": query_for_chain})

# 체인 실행 결과 출력
# 결과는 딕셔너리 형태로, 'input', 'context', 'answer' 키를 포함함.
# 'context'에는 retriever가 검색한 문서(Document 객체 리스트)가 들어있음.
# 'answer'에는 LLM이 생성한 최종 답변이 들어있음.
print("--- RAG Chain 전체 응답 --- ")
print(response_from_chain)

print("\n--- RAG Chain 응답 키 --- ")
print(response_from_chain.keys())

print("\n--- RAG Chain 최종 답변 (answer) --- ")
print(response_from_chain["answer"])

--- RAG Chain 전체 응답 --- 
{'input': '리비안 창업자는 누구이고, 어떤 회사로부터 투자를 받았나요?', 'context': [Document(metadata={'source': '../data\\Rivian_KR.txt'}, page_content='리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.\n2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.'), Document(metadata={'source': '../data\\Rivian_KR.txt'}, page_content='리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.\n2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.')], 'answer': '죄송합니다, 제공된 정보로는 답변을 찾을 수 없습니다.'}

--- RAG Chain 응답 키 --- 
dict_keys(['input', 'context', 'answer'])

--- RAG Chain 최종 답변 (answer) --- 
죄송합니다, 제공된 정보로는 답변을 찾을 수 없습니다.


**`[추가 설명] create_stuff_documents_chain` 자세히 보기**

`create_stuff_documents_chain`은 이름처럼 검색된 문서들을 'stuff'(채워넣기) 방식으로 처리하는 체인임.
얘는 독립적으로 사용될 수도 있음. 즉, Retriever 없이 미리 준비된 문서들을 컨텍스트로 제공하고 질문에 답변하게 할 수 있음.

**작동 방식:**
1.  입력으로 `input`(사용자 질문)과 `context`(참고 문서 Document 객체 리스트)를 받음.
2.  이 `input`과 `context` (문서들의 `page_content`가 합쳐짐)를 지정된 프롬프트 템플릿에 채워 넣음.
3.  완성된 프롬프트를 LLM에게 전달하여 답변을 생성함.
4.  LLM의 답변을 반환함 (이때는 `answer` 키가 아니라 바로 답변 문자열이 나옴).

In [19]:
from langchain_core.documents import Document # Document 객체를 직접 만들 때 필요

# 위에서 만든 combine_docs_chain (create_stuff_documents_chain의 결과물)을 재사용

# 참고할 문서를 직접 Document 객체 리스트로 만들어줌
manual_documents_for_stuff = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 강력한 프레임워크입니다.", metadata={"source": "manual_doc_1"}),
    Document(page_content="LangChain은 파이썬과 타입스크립트(자바스크립트)를 주로 지원합니다.", metadata={"source": "manual_doc_2"})
]

response_from_stuff_chain_direct = combine_docs_chain.invoke({
    "input": "LangChain은 어떤 프로그래밍 언어를 지원하나요?",
    "context": manual_documents_for_stuff # Retriever 대신 직접 문서를 context로 제공
})

# create_stuff_documents_chain의 직접적인 출력은 LLM의 답변 문자열임.
print("--- create_stuff_documents_chain 직접 호출 결과 --- ")
print(response_from_stuff_chain_direct)

--- create_stuff_documents_chain 직접 호출 결과 --- 
LangChain은 주로 파이썬과 타입스크립트(자바스크립트)를 지원합니다.


**`[추가 설명] create_retrieval_chain` 자세히 보기**

`create_retrieval_chain`은 두 개의 주요 컴포넌트(또는 그 역할을 하는 체인)를 결합함:
1.  **Retriever:** 사용자 질문(`input`)을 받아 관련 문서를 검색하는 역할. (필수 인자: `retriever`)
2.  **Document Combining Chain (예: `combine_docs_chain`):** 검색된 문서(`context`)와 사용자 질문(`input`)을 받아 최종 답변(`answer`)을 생성하는 역할. (필수 인자: `combine_docs_chain`)

이 함수는 이 두 단계를 자동으로 연결하여 전체 RAG 파이프라인을 쉽게 만들 수 있도록 도와줌.
Retriever 부분에는 `vectorstore.as_retriever()` 뿐만 아니라, 다른 종류의 Retriever 객체도 전달할 수 있음.

아래 코드는 `rag_chain`을 생성할 때 `retriever` 인자에 `vectorstore.as_retriever()`를 직접 사용하고, `search_kwargs`를 다르게 설정하여 다른 검색 결과를 얻는 예시임. (k=4로 설정)


In [20]:
# create_retrieval_chain에 retriever를 직접 생성해서 전달하는 예시
# vectorstore.as_retriever()를 호출할 때 search_kwargs를 다르게 설정하면 다른 검색 결과를 얻을 수 있음.
# 여기서는 k=4 (기본값)를 사용하도록 retriever를 새로 만듦.
alternative_retriever = vectorstore.as_retriever() # k의 기본값인 4개 문서를 가져옴

alternative_rag_chain = create_retrieval_chain(
    retriever=alternative_retriever, 
    combine_docs_chain=combine_docs_chain # 위에서 만든 stuff 체인 재사용
)

# 체인 실행 (동일한 질문 사용)
alternative_response = alternative_rag_chain.invoke({"input": query_for_chain})

print("--- 다른 Retriever 설정을 사용한 RAG Chain 응답 --- ")
print(f"입력 질문: {alternative_response['input']}")
print(f"검색된 컨텍스트 문서 수: {len(alternative_response['context'])}")
# for i, doc in enumerate(alternative_response['context']):
#     print(f"  컨텍스트 {i+1}: {doc.page_content[:50]}... (출처: {doc.metadata['source']})")
print(f"최종 답변: {alternative_response['answer']}")
# k=4로 설정했기 때문에, context에 더 많은 (또는 다른) 문서가 포함되어 답변에 영향을 줄 수 있음.

--- 다른 Retriever 설정을 사용한 RAG Chain 응답 --- 
입력 질문: 리비안 창업자는 누구이고, 어떤 회사로부터 투자를 받았나요?
검색된 컨텍스트 문서 수: 4
최종 답변: 죄송합니다, 제공된 정보로는 답변을 찾을 수 없습니다.



🧐실제 RAG 시스템을 구축할 때는🧐:
1. 🚀데이터 전처리🚀 가장 중요할 것으로 예상하며, 문서 로딩 후 추가적인 정제 작업이 필요할 수 있음. 
2. 분할 전략: `RecursiveCharacterTextSplitter`, 마크다운/코드용 스플리터 등 데이터 특성에 맞는 분할기 선택이 중요함.
3. 임베딩 모델 선택: 한국어 특화 모델, 오픈소스 모델 등 다양한 옵션 고려.
4. 벡터 저장소 선택: 데이터 규모, 성능 요구사항, 운영 환경에 따라 적합한 DB 선택.
5. 검색 전략 고도화:  MMR, 하이브리드 검색, 재순위(Re-ranking) 모델 적용 등.
6. 프롬프트 엔지니어링: LLM이 컨텍스트를 잘 활용하고 원하는 형태로 답변하도록 프롬프트를 정교하게 다듬어야 함.
7. 평가: RAG 시스템의 성능을 객관적으로 측정하고 개선하기 위한 평가 파이프라인 구축도 중요함.